## Deliverable 2. Create a Customer Travel Destinations Map.
Retrieving customer weather preferences to identify potential travel destinations and nearby hotels.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Ravar,IR,31.2656,56.8055,80.29,18,0,3.56,clear sky
1,Venice,US,27.0998,-82.4543,92.14,73,0,10.36,clear sky
2,Trinidad,BO,-14.8333,-64.9000,70.43,88,100,1.30,overcast clouds
3,Chulumani,BO,-16.4096,-67.5263,63.63,95,99,2.64,overcast clouds
4,Rikitea,PF,-23.1203,-134.9692,69.26,68,92,22.01,overcast clouds


In [31]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
try:
    min_temp = int(input("Please enter your minimum temperature preference: "))
except ValueError:
    min_temp = int(input("Please enter a whole number for the minimum temperature: "))
try:
    max_temp = int(input("Please enter your maximum temperature preference: "))
except ValueError:
    max_temp = int(input("Please enter a whole number for the maximum temperature: "))

In [39]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp) ]
clean_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Ravar,IR,31.2656,56.8055,80.29,18,0,3.56,clear sky
1,Venice,US,27.0998,-82.4543,92.14,73,0,10.36,clear sky
5,Kavieng,PG,-2.5744,150.7967,81.95,74,34,8.25,scattered clouds
6,Hithadhoo,MV,-0.6000,73.0833,81.30,76,59,9.80,broken clouds
14,Dakar,SN,14.6937,-17.4441,81.48,81,75,9.82,broken clouds


In [40]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()


City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No empty rows shown

In [42]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ravar,IR,80.29,clear sky,31.2656,56.8055,
1,Venice,US,92.14,clear sky,27.0998,-82.4543,
5,Kavieng,PG,81.95,scattered clouds,-2.5744,150.7967,
6,Hithadhoo,MV,81.30,broken clouds,-0.6000,73.0833,
14,Dakar,SN,81.48,broken clouds,14.6937,-17.4441,


In [68]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for i, row in hotel_df.iterrows():


    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [77]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].dropna(inplace=True)
clean_hotel_df = hotel_df.copy()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ravar,IR,80.29,clear sky,31.2656,56.8055,computertik
1,Venice,US,92.14,clear sky,27.0998,-82.4543,A Beach Retreat On Casey Key
5,Kavieng,PG,81.95,scattered clouds,-2.5744,150.7967,Nusa Island Retreat
6,Hithadhoo,MV,81.30,broken clouds,-0.6000,73.0833,Scoop Guest House
14,Dakar,SN,81.48,broken clouds,14.6937,-17.4441,Novotel Dakar


In [78]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [84]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [85]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))